In [ ]:
from dask_gateway import GatewayCluster
import dask.distributed
import dask.utils
import dask.array
import planetary_computer
from pystac_client import Client
import odc.stac
from shapely.geometry import Polygon
import geopandas
import numpy
import xarray
import rasterio
import rasterio.enums
import gc
import math
from tqdm.notebook import trange, tqdm
import os
from pbpcu.sentinel2 import apply_sen2_offset

In [ ]:
def apply_sen2_vld_msk(scns_xa, bands, qa_pxl_msk="SCL", out_no_data_val=0):
    scns_lcl_xa = scns_xa.copy()
    for band in bands:
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 0] = out_no_data_val  # No Data
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 1] = out_no_data_val  # Saturation
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 2] = out_no_data_val  # Cast Shadow
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 3] = out_no_data_val  # Cloud Shadows
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 8] = out_no_data_val  # Cloud Medium Probability
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 9] = out_no_data_val  # Cloud High Probability
        scns_lcl_xa[band].values[scns_lcl_xa["SCL"].values == 10] = out_no_data_val # Thin Cirrus
    return scns_lcl_xa

def get_img_metadata(img_file):
    img_data_obj = rasterio.open(img_file)
    img_bounds = img_data_obj.bounds
    img_bbox = [img_bounds.left, img_bounds.bottom, img_bounds.right, img_bounds.top]
    img_x_res, img_y_res  = img_data_obj.res
    if img_y_res > 0:
        img_y_res = img_y_res * (-1)
    img_data_obj = None
    return img_bbox, img_x_res, img_y_res

def get_img_band_array(img_file, band=1):
    img_data_obj = rasterio.open(img_file)
    img_arr = img_data_obj.read(band)
    img_data_obj = None
    return img_arr

def find_month_end_date(year, month):
    import calendar
    cal = calendar.Calendar()
    month_days = cal.monthdayscalendar(year, month)
    max_day_month = numpy.array(month_days).flatten().max()
    return max_day_month

def zero_pad_num_str(
    num_val: float,
    str_len: int = 3,
    round_num: bool = False,
    round_n_digts: int = 0,
    integerise: bool = False,
    absolute: bool = False,
    gain: float = 1,
) -> str:
    if absolute:
        num_val = abs(num_val)
    if round_num:
        num_val = round(num_val, round_n_digts)
    if integerise:
        num_val = int(num_val * gain)

    num_str = "{}".format(num_val)
    num_str = num_str.zfill(str_len)
    return num_str

In [ ]:
def find_recent_alerts(tile, start_year, end_year, c_year, c_month, out_dir):
    years = numpy.arange(start_year, end_year, 1)
    found_tile_alerts = False
    found_miss_alerts = False
    meta_img_file = ""
    alerts_img_file = ""
    alerts_year = 0
    alerts_month = 0
    for year in years:
        for month_idx in numpy.arange(0, 12, 1):
            month = month_idx + 1
            month_end_day = find_month_end_date(year, month)
            month_str = zero_pad_num_str(month, str_len = 2, round_num = False, round_n_digts = 0)
            month_end_day_str = zero_pad_num_str(month_end_day, str_len = 2, round_num = False, round_n_digts = 0)
            date_str = f"{year}{month_str}"
            if (year == c_year) and (month == c_month):
                # Cannot process beyond current month of current year as there will be no data - its the future...
                break

            out_meta_img_file = os.path.join(out_dir, f'gmw_{tile}_{date_str}_chg_alerts_meta.tif')
            out_alerts_img_file = os.path.join(out_dir, f'gmw_{tile}_{date_str}_chg_alerts.tif')
            
            if os.path.exists(out_meta_img_file) and os.path.exists(out_alerts_img_file):
                found_tile_alerts = True
                meta_img_file = out_meta_img_file
                alerts_img_file = out_alerts_img_file
                alerts_year = year
                alerts_month_idx = month_idx
            else:
                found_miss_alerts = True
                break
        if found_miss_alerts:
            break
    
    if found_tile_alerts:
        start_month_idx = alerts_month_idx + 1
        if start_month_idx == 12:
            start_month_idx = 0
            alerts_year = alerts_year + 1
        years = numpy.arange(alerts_year, end_year, 1)
        score_arr = get_img_band_array(meta_img_file, band=1)
        date_year_arr = get_img_band_array(meta_img_file, band=2)
        date_month_arr = get_img_band_array(meta_img_file, band=3)
    else:
        score_arr = numpy.zeros_like(gmw_ext_msk, dtype=numpy.int16)
        date_year_arr = numpy.zeros_like(gmw_ext_msk, dtype=numpy.uint16)
        date_month_arr = numpy.zeros_like(gmw_ext_msk, dtype=numpy.uint8)
        start_month_idx = 0
    
    return years, start_month_idx, score_arr, date_year_arr, date_month_arr

In [ ]:
cluster = GatewayCluster()  # Creates the Dask Scheduler. Might take a minute.
cluster.adapt(minimum=4, maximum=24)
print(cluster.dashboard_link)

client = dask.distributed.Client(cluster, timeout=5)
odc.stac.configure_rio(cloud_defaults=True, client=client)

In [ ]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [ ]:
bands = ["B04", "B08", "SCL"]

In [ ]:
## Testing Tiles:
#tiles = ["S13E047", "S15E046", "S17E036", "S19E034", "S07E039", "N12W016", "N10W014", "N14W016", "N06E004", "N06E005", "N05E007", "N01E009", "N06E000"]


# All of Africa Tiles:
#tiles = ["S10E039", "S25E044", "S13E049", "S10E040", "N05E007", "N22E039", "N05E008", "N22E036", "S28E032", "N29E034", "N21E037", "S25E032", "N27E035", "N11W016", "N12W017", "N14E042", "S16E045", "S18E044", "S10E047", "S09E013", "S07E012", "S18E035", "S13E047", "S30E030", "N17E042", "S16E044", "S15E050", "S15E045", "N14E043", "S07E013", "N06E000", "N09W014", "S02E010", "N27E034", "S26E032", "N24E035", "N06E008", "N05E009", "N12W016", "S03E039", "S13E040", "N07W012", "N21E039", "N09W013", "N05E006", "S03E040", "N27E033", "S13E048", "N21E040", "N19E037", "N06E005", "N24E038", "N11E045", "S08E039", "N12E051", "S06E038", "S15E040", "S05E039", "S16E049", "S20E044", "S01E009", "S16E038", "S20E035", "S18E036", "S15E047", "S16E046", "N17E040", "N19E041", "N12E043", "N11W015", "S20E043", "N17W017", "N14W016", "S28E031", "N19E038", "N03E009", "N17E039", "N00E008", "N27E036", "N14W017", "N12W015", "S20E057", "N11W014", "S23E043", "S05E053", "N00E009", "S04E011", "S33E027", "N17E041", "S15E046", "S15E049", "S23E035", "S20E034", "S16E039", "S02E009", "S01E008", "N07W011", "N06E004", "N05E005", "S05E038", "S06E039", "S16E040", "N12E050", "N07E003", "N05W002", "N06W003", "S17E036", "S12E044", "N13E042", "S32E027", "N15E040", "N15W017", "S22E043", "N10W014", "N13W015", "N01E009", "N26E037", "S29E030", "S09E046", "N06W004", "N06W011", "N01E006", "N05W010", "N07E004", "S07E039", "N15W018", "S12E043", "S31E029", "S32E028", "S21E034", "S01E042", "S22E035", "S17E049", "S14E048", "S32E029", "S17E038", "S04E055", "S21E035", "N07E005", "N06W010", "N06W005", "N25E038", "S09E039", "S11E043", "S04E039", "S14E040", "S21E043", "N10W015", "S10E013", "N16W017", "N18E038", "S09E047", "N00E042", "S29E031", "N23E035", "N26E036", "N25E037", "N16E039", "N02E009", "S17E037", "N06W002", "N05W003", "N07E002", "N16E040", "S14E047", "N18E041", "S12E045", "N13E043", "N25E035", "N05W009", "N26E034", "S27E032", "N04E009", "N28E035", "S22E040", "N13W016", "N13E054", "S31E030", "N16E042", "S17E044", "N08W014", "S19E034", "S05E012", "S03E010", "S02E040", "S01E041", "N20E040", "S12E048", "S11E049", "S17E043", "S12E040", "N13E053", "S19E057", "N20W017", "N05W006", "N23E038", "N08W013", "S11E040", "N08W012", "N23E039", "N05W007", "N06W006", "N28E033", "S01E040", "S02E041", "S24E044", "S12E049", "N15E042", "S19E044", "N18E042", "N06W001", "S03E011", "S08E013", "S06E012", "S19E035", "N07E001", "N20E037", "N00E041", "N23E036", "N04E008", "N05W008", "N06W009", "N13W017"]
#tiles = tiles[0:50]

# All 2022 Region Tiles
tiles = ["S05W081", "N01W081", "N02W080", "S03W081", "N02W079", "S03W080", "N01W080", "S02W080", "S04W082", "N00W081", "S02W081", "N22W091", "N19W104", "N19W089", "N17W095", "N19W094", "N21W098", "N21W097", "N19W103", "N22W106", "N19W093", "N27W110", "N19W092", "N21W106", "N22W098", "N19W095", "N17W094", "N19W088", "N21W091", "N19W105", "N22W090", "N17W096", "N27W114", "N24W098", "N19W097", "N22W087", "N24W108", "N22W088", "N29W112", "N27W113", "N24W107", "N17W101", "N17W099", "N17W098", "N17W100", "N27W112", "N21W088", "N29W113", "N19W091", "N22W089", "N21W087", "N19W096", "N29W114", "N27W098", "N26W112", "N25W113", "N20W105", "N20W088", "N28W113", "N18W101", "N16W098", "N28W114", "N26W098", "N16W097", "N25W109", "N20W092", "N25W108", "N26W109", "N16W096", "N28W112", "N25W112", "N26W113", "N30W113", "N20W096", "N18W102", "N16W093", "N20W106", "N28W110", "N23W107", "N30W114", "N23W090", "N20W091", "N23W098", "N16W094", "N23W099", "N28W111", "N23W106", "N25W111", "N15W093", "N26W110", "N18W103", "N20W097", "N06W056", "N06W057", "N06W058", "N05W053", "N06W055", "N06W054", "N05W052", "N06W053", "N04W052", "N07W057", "N04W051", "N07W058", "N07W056", "N08W060", "N09W060", "N08W059", "N07W059", "N23E091", "N18E094", "N16E095", "N10E098", "N13E098", "N16E094", "N13E097", "N23E090", "N18E097", "N15E097", "N20E093", "N23E087", "N23E088", "N20E094", "N15E098", "N23E089", "N16E097", "N22E089", "N17E097", "N21E092", "N22E087", "N22E092", "N21E093", "N17E096", "N14E098", "N22E088", "N12E098", "N22E090", "N17E094", "N19E094", "N11E097", "N17E095", "N22E091", "N19E093", "N11E098", "N02E120", "N01E121", "S05E123", "N04E125", "S03E121", "S04E119", "S03E120", "S05E122", "N01E120", "N02E121", "N01E122", "N00E119", "S05E120", "S03E122", "N02E124", "S02E119", "S03E123", "N02E125", "N01E124", "S05E121", "S06E120", "S01E119", "S02E118", "N01E123", "N02E122", "S01E122", "S02E123", "N03E125", "S04E121", "S03E118", "N00E123", "N00E122", "N01E119", "S04E120", "S02E122", "S01E123", "S03E119", "S05E119", "S01E121", "S02E120", "S04E122", "N00E120", "S05E117", "N00E121", "S05E118", "S04E123", "S02E121", "S01E120", "N06E118", "S01E109", "N06E117", "S02E112", "N08E116", "S02E113", "N08E117", "N05E117", "N06E116", "N00E109", "S04E116", "N03E112", "S01E108", "N05E118", "N06E119", "S02E116", "S04E114", "N03E118", "N05E115", "S01E110", "N00E116", "N03E117", "S02E111", "S02E110", "N00E117", "N05E114", "N06E115", "S04E115", "S01E116", "N03E111", "N01E117", "S03E110", "N04E113", "N02E111", "N01E118", "N02E118", "S03E116", "N02E110", "N02E117", "S03E111", "N02E108", "N01E109", "S03E113", "N07E116", "N04E117", "S03E114", "S03E115", "N07E118", "N07E117", "S03E112", "N01E108", "N02E109", "S10E039", "S25E044", "S13E049", "S10E040", "N05E007", "N05E008", "S28E032", "S25E032", "N11W016", "N12W017", "S16E045", "S18E044", "S10E047", "S09E013", "S07E012", "S18E035", "S13E047", "S30E030", "S16E044", "S15E050", "S15E045", "S07E013", "N06E000", "N09W014", "S02E010", "S26E032", "N06E008", "N05E009", "N12W016", "S03E039", "S13E040", "N07W012", "N09W013", "N05E006", "S03E040", "S13E048", "N06E005", "S08E039", "S06E038", "S15E040", "S05E039", "S16E049", "S20E044", "S01E009", "S16E038", "S20E035", "S18E036", "S15E047", "S16E046", "N11W015", "S20E043", "N17W017", "N14W016", "S28E031", "N03E009", "N00E008", "N14W017", "N12W015", "N11W014", "S23E043", "N00E009", "S04E011", "S33E027", "S15E046", "S15E049", "S23E035", "S20E034", "S16E039", "S02E009", "S01E008", "N07W011", "N06E004", "N05E005", "S05E038", "S06E039", "S16E040", "N07E003", "N05W002", "N06W003", "S17E036", "S12E044", "S32E027", "N15W017", "S22E043", "N10W014", "N13W015", "N01E009", "S29E030", "S09E046", "N06W004", "N06W011", "N01E006", "N05W010", "N07E004", "S07E039", "N15W018", "S12E043", "S31E029", "S32E028", "S21E034", "S01E042", "S22E035", "S17E049", "S14E048", "S32E029", "S17E038", "S21E035", "N07E005", "N06W010", "N06W005", "S09E039", "S11E043", "S04E039", "S14E040", "S21E043", "N10W015", "S10E013", "N16W017", "S09E047", "N00E042", "S29E031", "N02E009", "S17E037", "N06W002", "N05W003", "N07E002", "S14E047", "S12E045", "N05W009", "S27E032", "N04E009", "S22E040", "N13W016", "S31E030", "S17E044", "N08W014", "S19E034", "S05E012", "S03E010", "S02E040", "S01E041", "S12E048", "S11E049", "S17E043", "S12E040", "N20W017", "N05W006", "N08W013", "S11E040", "N08W012", "N05W007", "N06W006", "S01E040", "S02E041", "S24E044", "S12E049", "S19E044", "N06W001", "S03E011", "S08E013", "S06E012", "S19E035", "N07E001", "N00E041", "N04E008", "N05W008", "N06W009", "N13W017", "N22E039", "N22E036", "N29E034", "N21E037", "N27E035", "N14E042", "N17E042", "N14E043", "N27E034", "N24E035", "N21E039", "N27E033", "N21E040", "N19E037", "N24E038", "N11E045", "N12E051", "N17E040", "N19E041", "N12E043", "N19E038", "N17E039", "N27E036", "S20E057", "S05E053", "N17E041", "N12E050", "N13E042", "N15E040", "N26E037", "S04E055", "N25E038", "N18E038", "N23E035", "N26E036", "N25E037", "N16E039", "N16E040", "N18E041", "N13E043", "N25E035", "N26E034", "N28E035", "N13E054", "N16E042", "N20E040", "N13E053", "S19E057", "N23E038", "N23E039", "N28E033", "N15E042", "N18E042", "N20E037", "N23E036", "S19E178", "S17E179", "S18W180", "S17E176", "S17E177", "S16W180", "S18W179", "S17E178", "S19E179", "S17W180", "S16E177", "S16E178", "S18E179", "S18E178", "S17W179", "S16E179", "S18E177"]
#tiles = tiles[0:50]

# Missing:  "N29W113"
#tiles.reverse()

n_tiles = len(tiles)

In [ ]:
n_tiles

In [ ]:
out_dir = "../monthly_change_imgs_v8"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [ ]:
start_year = 2019
end_year = 2023

In [ ]:
c_year = 2022
c_month = 11

In [ ]:
n_tile = 0
for tile in tiles:
    print(f"{tile}: ({n_tile+1} of {n_tiles})")
    
    # Path of the revised GMW 2018 extent image.
    gmw_tile_img = f"../gmw_2018_revised_ext_opt_s1/gmw_{tile}_2018_alerts_ext.tif"
    
    # If the GMW 2018 tile does not exist then skip tile and go to the next one.
    if not os.path.exists(gmw_tile_img):
        print("\tNo GMW Extent Tile Available...")
        n_tile += 1
        continue
    
    # Get image tile meta-data
    bbox, img_x_res, img_y_res = get_img_metadata(gmw_tile_img)
    # Get GMW Mask as array
    gmw_ext_msk = get_img_band_array(gmw_tile_img, band=2)
    
    # Check there are mangrove pixels within the scene (mangroves == 1)
    if numpy.max(gmw_ext_msk) == 0:
        print("\tNo Mangrove Pixels in Mask...")
        del gmw_ext_msk
        n_tile += 1
        continue

    # Get the image shape (i.e., number of pixels)
    img_shp = gmw_ext_msk.shape

    # Define the output image spatial transformation.
    out_img_transform = rasterio.transform.Affine(img_x_res, 0.0, bbox[0], 0.0, img_y_res, bbox[3])
    
    # Find when the latest alerts have been created and continue from any previous alerts
    # This is required for if the analysis crashes or when rerun each month for new alerts
    years, start_month_idx, score_arr, date_year_arr, date_month_arr = find_recent_alerts(tile, start_year, end_year, c_year, c_month, out_dir)
    print(f"\tYears: {years}; Start Month Index: {start_month_idx}")
    
    # Skip to the next tile if the tile alerts for the tile have already been processed.
    if (len(years) == 1) and (years[0] == c_year) and (start_month_idx == c_month-1):
        n_tile += 1
        continue
    
    # Iterate through the years for the analysis
    for year in years:
        # Date range of the ROI (Beginning of Jan to end of Dec)
        chng_time_range = f"{year}-01-01/{year}-12-31"
        
        ####################################################################
        # Find the scenes for the year of interest.
        chng_search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox, datetime=chng_time_range, query={"eo:cloud_cover": {"lt": 50}})
        chng_items = chng_search.get_all_items()
        n_chng_items = len(chng_items)
        print(f"\t{chng_time_range}: N Items = {n_chng_items}")
        ####################################################################

        ####################################################################
        # Read the scenes into dask array structure and make persistant in memory
        signed_chng_items = [planetary_computer.sign(item) for item in chng_items]

        sen2_chng_scn_xa = odc.stac.stac_load(
            signed_chng_items,
            bands=bands,
            groupby="solar_day",
            chunks={"time":12, "latitude": 1024, "longitude": 1024},
            bbox=bbox,
            crs="EPSG:4326",
            resolution=img_x_res
        )

        # Comment out for larger datasets which don't fit into memory.
        sen2_chng_scn_xa = sen2_chng_scn_xa.persist()
        
        # Apply Offset
        sen2_chng_scn_xa = apply_sen2_offset(sen2_chng_scn_xa)
        
        # Apply cloud mask to all scenes
        sen2_chng_scn_xa = sen2_chng_scn_xa.map_blocks(apply_sen2_vld_msk, kwargs={"bands":bands})
        ####################################################################

        ####################################################################
        # 'Clean' up the Red and NIR bands to remove any values less than zero."
        sen2_chng_scn_xa['B04'] = sen2_chng_scn_xa.B04.where(sen2_chng_scn_xa.B04>0)
        sen2_chng_scn_xa['B08'] = sen2_chng_scn_xa.B08.where(sen2_chng_scn_xa.B08>0)

        # Valid pixel mask
        sen2_chng_vld_xa = sen2_chng_scn_xa.B04>0

        # Calculate the NDVI
        ndvi_chng_scn_xa = ((sen2_chng_scn_xa.B08-sen2_chng_scn_xa.B04)/(sen2_chng_scn_xa.B08+sen2_chng_scn_xa.B04))
        ####################################################################


        ####################################################################
        # Date range of the year before ROI for checking changes have changed over the last 12 months.
        ref_time_range = f"{year-1}-01-01/{year-1}-12-31"
        # Search for reference scenes from year earlier than ROI.
        ref_search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox, datetime=ref_time_range, query={"eo:cloud_cover": {"lt": 50}})
        ref_items = ref_search.get_all_items()
        n_ref_items = len(ref_items)
        print(f"\t{ref_time_range}: N Items = {n_ref_items}")
        ####################################################################

        ####################################################################
        # Read the scenes into dask array structure and make persistant in memory
        signed_ref_items = [planetary_computer.sign(item) for item in ref_items]

        sen2_ref_scn_xa = odc.stac.stac_load(
            signed_ref_items,
            bands=bands,
            groupby="solar_day",
            chunks={"time":12, "latitude": 1024, "longitude": 1024},
            bbox=bbox,
            crs="EPSG:4326",
            resolution=img_x_res
        )

        # Comment out for larger datasets which don't fit into memory.
        sen2_ref_scn_xa = sen2_ref_scn_xa.persist()
        
        # Apply Offset
        sen2_ref_scn_xa = apply_sen2_offset(sen2_ref_scn_xa)
        
        # Apply cloud mask to all scenes
        sen2_ref_scn_xa = sen2_ref_scn_xa.map_blocks(apply_sen2_vld_msk, kwargs={"bands":bands})
        ####################################################################

        ####################################################################
        # 'Clean\' up the Red and NIR bands to remove any values less than zero."
        sen2_ref_scn_xa['B04'] = sen2_ref_scn_xa.B04.where(sen2_ref_scn_xa.B04>0)
        sen2_ref_scn_xa['B08'] = sen2_ref_scn_xa.B08.where(sen2_ref_scn_xa.B08>0)

        # Calculate the NDVI
        ndvi_ref_scn_xa = ((sen2_ref_scn_xa.B08-sen2_ref_scn_xa.B04)/(sen2_ref_scn_xa.B08+sen2_ref_scn_xa.B04))

        # Create monthly summaries
        monthly_ref_ndvi_xa = ndvi_ref_scn_xa.resample(time='1MS').median(skipna=True).compute()
        ####################################################################
        
        # If first year start at the starting month which might not be Jan.
        if years[0] == year:
            months = numpy.arange(start_month_idx, 12, 1)
        else:
            months = numpy.arange(0, 12, 1)
        
        # Iterate through the months...
        for month_idx in months:
            month = month_idx + 1
            month_end_day = find_month_end_date(year, month)
            month_str = zero_pad_num_str(month, str_len = 2, round_num = False, round_n_digts = 0)
            month_end_day_str = zero_pad_num_str(month_end_day, str_len = 2, round_num = False, round_n_digts = 0)
            date_str = f"{year}{month_str}"
            print(f"\t\t{year} - {month_str}")
            if (year == c_year) and (month == c_month):
                # Cannot process beyond current month of current year as there will be no data - its the future...
                break
            
            # If there isn't data for a month then set index for last month available so index is within bounds.
            if month_idx >= len(monthly_ref_ndvi_xa.time):
                month_idx = len(monthly_ref_ndvi_xa.time)-1
            
            # Output file names for the change alerts and associated metadata (i.e., score and date of first change observation).
            out_meta_img_file = os.path.join(out_dir, f'gmw_{tile}_{date_str}_chg_alerts_meta.tif')
            out_alerts_img_file = os.path.join(out_dir, f'gmw_{tile}_{date_str}_chg_alerts.tif')
            
            # Take a time slice of the per-scene NDVIs for the current month
            ndvi_month_chng_scn_xa = ndvi_chng_scn_xa.sel(time=slice(f"{year}-{month_str}-01", f"{year}-{month_str}-{month_end_day_str}"))
            # Find the pixels of change for the month on a per-scene basis (i.e., NDVI < 0.25 and then difference in NDVI from a year earlier is > 0.15)
            monthly_chng_pxls_xa = numpy.logical_and((ndvi_month_chng_scn_xa < 0.25), numpy.abs(ndvi_month_chng_scn_xa - monthly_ref_ndvi_xa.values[month_idx]) > 0.15)
            
            # Get the valid masks for the current month
            monthly_vld_pxls_scn_xa = sen2_chng_vld_xa.sel(time=slice(f"{year}-{month_str}-01", f"{year}-{month_str}-{month_end_day_str}")) 

            # Mask the GMW changes and valid masks to the GMW mask.
            gmw_monthly_chng_pxls_xa = monthly_chng_pxls_xa.where(gmw_ext_msk == 1)
            gmw_monthly_vld_pxls_scn_xa = monthly_vld_pxls_scn_xa.where(gmw_ext_msk == 1)

            # Sum the number of changes observed in the scenes
            gmw_monthly_chng_pxls_count_xa = gmw_monthly_chng_pxls_xa.sum(dim="time", skipna=True)
            # Sum the number of observations.
            gmw_monthly_vld_pxls_count_xa = gmw_monthly_vld_pxls_scn_xa.sum(dim="time", skipna=True)
            
            # Run the compute in dask - before this no analysis actually happens it is just building the analysis graph
            dask.compute(gmw_monthly_chng_pxls_count_xa, gmw_monthly_vld_pxls_count_xa)

            ####################################################################
            # Update the score data values (Add 1 for a change and -1 for no change).
            lcl_score_arr = score_arr + (gmw_monthly_chng_pxls_count_xa.values) - (gmw_monthly_vld_pxls_count_xa.values - gmw_monthly_chng_pxls_count_xa.values)
            # Any pixels with a score of 5 before the analysis set back to 5 - they have already been identified as a change.
            lcl_score_arr[score_arr == 5] = 5

            # Update the date layers so that any pixels which have gone back to a score of 0 are reset.
            date_year_arr[lcl_score_arr < 1] = 0
            date_month_arr[lcl_score_arr < 1] = 0

            # For pixels with a score > 0 without a date define, define the date as this is the first observation.
            date_year_arr[(lcl_score_arr > 0) & (date_year_arr == 0)] = year
            date_month_arr[(lcl_score_arr > 0) & (date_month_arr == 0)] = month

            # Identify new confirmed alerts (i.e., the score has now reach 5).
            month_alerts_arr = numpy.zeros_like(score_arr, dtype=numpy.uint8)
            month_alerts_arr[numpy.logical_and(score_arr<5, lcl_score_arr>4)] = 1

            # Cap the score layer at 5 and don't allow scores lower than 0.
            lcl_score_arr[lcl_score_arr>5] = 5
            lcl_score_arr[lcl_score_arr<1] = 0

            # Copy the local score image to replace the score image
            score_arr = numpy.copy(lcl_score_arr)
            ####################################################################

            ####################################################################
            # Create the output image file for the metadata
            with rasterio.open(out_meta_img_file,
                                            'w',
                                            driver='COG',
                                            height=img_shp[0],
                                            width=img_shp[1],
                                            count=3,
                                            dtype=numpy.uint16,
                                            crs='epsg:4326',
                                            transform=out_img_transform,
                                        ) as out_img_dataset:

                # Write output array to the image file
                out_img_dataset.write(score_arr, 1)
                # Name the image band.
                out_img_dataset.set_band_description(1, "Alerts_Score")

                # Write output array to the image file
                out_img_dataset.write(date_year_arr, 2)
                # Name the image band.
                out_img_dataset.set_band_description(2, "Alert_Date_Year")

                # Write output array to the image file
                out_img_dataset.write(date_month_arr, 3)
                # Name the image band.
                out_img_dataset.set_band_description(3, "Alert_Date_Month")
            ####################################################################


            ####################################################################
            # Create the output image file for the newly identified alerts.
            with rasterio.open(out_alerts_img_file,
                                            'w',
                                            driver='COG',
                                            height=img_shp[0],
                                            width=img_shp[1],
                                            count=1,
                                            dtype=numpy.uint8,
                                            crs='epsg:4326',
                                            transform=out_img_transform,
                                        ) as out_img_dataset:

                # Write output array to the image file
                out_img_dataset.write(month_alerts_arr, 1)
                # Name the image band.
                out_img_dataset.set_band_description(1, "Alerts")
            ####################################################################
            # Delete Monthly Objects.
            del ndvi_month_chng_scn_xa
            del monthly_chng_pxls_xa
            del monthly_vld_pxls_scn_xa
            del gmw_monthly_chng_pxls_xa
            del gmw_monthly_vld_pxls_scn_xa
            del gmw_monthly_chng_pxls_count_xa
            del gmw_monthly_vld_pxls_count_xa
            del lcl_score_arr
            del month_alerts_arr
        # Delete annual objects
        del sen2_chng_scn_xa
        del sen2_chng_vld_xa
        del ndvi_chng_scn_xa
        del sen2_ref_scn_xa
        del ndvi_ref_scn_xa
        del monthly_ref_ndvi_xa
    # Delete tile based arrays
    del score_arr
    del date_year_arr
    del date_month_arr
    del gmw_ext_msk
    # Increment the tile number for user feedback.
    n_tile += 1
    # Restart the dask workers to ensure all the memory etc. is cleared.
    client.restart(wait_for_workers=False)

In [ ]:
# Close the dask cluster
client.close()
cluster.close()